In [7]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [8]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

In [9]:
#imports
import os
import time
import pandas as pd
import numpy as np
import sys
from io import StringIO
import glob
if not hasattr(sys, "version_info") or sys.version_info < (3, 5):
  raise SystemExit("This program requires Python 3.5 or later.")
import csv
import matplotlib
#matplotlib.use('nbagg')
import matplotlib.pyplot as plt
## https://github.com/ipython/ipyparallel
import ipyparallel as ipp
from IPython.display import JSON
import json
import statistics as st
from pympler import asizeof
from hurry.filesize import size
import tabletext
import sqlite3
import shutil

In [10]:
def mstime():
    import time
    return round(time.time() * 1000)

def sync():
    import os
    os.system('sync')
    
def touch(path):
    import os
    with open(path, 'a'):
        os.utime(path, None)
    sync()
    
def unic(l: list) -> list :
    return list(set(l))

def cleanupTmp():
    print("clean tmp in %s" % cutPathes(tmpFolder))
    if tmpFolder == "/tmp":
        print("invalid tmp folder!")
        exit(-1)
    for filename in os.listdir(tmpFolder):
        file_path = os.path.join(tmpFolder, filename)
        # print("remove %s" % file_path)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (cutPathes(file_path), e))

In [11]:
# read env
envHome = os.environ.get('HOME')
envRamHome = os.environ.get('RAMCASHE')

#init env
tmpFolder = "%s/tmp" % envRamHome
if os.path.isdir(tmpFolder):
    shutil.rmtree(tmpFolder, ignore_errors=True)

os.mkdir(tmpFolder)

In [12]:
data_sources = glob.glob("data/*_shops.xml")
plt.rcParams['figure.figsize'] = [30/2.54, 20/2.54]

# tags
client = ipp.Client()
workers = client[:]

midDb = "%s/midData/mid.db" % os.getcwd()
midDbExample = "%s/midData/mid.example.db" % os.getcwd()
midDbTmp = "%s/midtmp.%d.db" % (os.getcwd(), mstime())
baseDir = os.getcwd() + "/"

def cutPathes(s: str) -> str:
    if s.startswith(baseDir):
        return s[len(baseDir)::]
    if s.startswith(envRamHome):
        return s[len(envRamHome)::]
    return s

print("engines count: %d" % len(client.ids))
print("source files count: %d" % len(data_sources))
print(tmpFolder)

engines count: 4
source files count: 2751
/home/ed/ramcashe/tmp


In [13]:
workers["tmpFolder"] = tmpFolder
workers["dbquery_cratetable"] = '''
CREATE TABLE "ttids" (
	"rowid"	INTEGER NOT NULL UNIQUE,
	"shapshotid"	TEXT NOT NULL,
	"ttid"	INTEGER NOT NULL,
	"buy_list"	TEXT NOT NULL,
	"sell_list"	TEXT NOT NULL,
	"capacity"	INTEGER NOT NULL,
	"buystat_avg"	REAL NOT NULL,
	"buystat_min"	REAL NOT NULL,
	"buystat_max"	REAL NOT NULL,
	"buystat_median"	REAL NOT NULL,
	"buystat_gap"	REAL NOT NULL,
	"buystat_stdev"	REAL NOT NULL,
	"sellstat_avg"	REAL NOT NULL,
	"sellstat_min"	REAL NOT NULL,
	"sellstat_max"	REAL NOT NULL,
	"sellstat_median"	REAL NOT NULL,
	"sellstat_gap"	REAL NOT NULL,
	"sellstat_stdev"	REAL NOT NULL,
	PRIMARY KEY("rowid" AUTOINCREMENT)
);'''
dbquery_cratetable = workers["dbquery_cratetable"][0]

workers["dbquery_insert"] = '''
INSERT INTO 'ttids'(
"shapshotid",
"ttid",
"buy_list",
"sell_list",
"capacity",
"buystat_avg",
"buystat_min",
"buystat_max",
"buystat_median",
"buystat_gap",
"buystat_stdev",
"sellstat_avg",
"sellstat_min",
"sellstat_max",
"sellstat_median",
"sellstat_gap",
"sellstat_stdev"
) VALUES (
?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);'''
dbquery_insert = workers["dbquery_insert"][0]

In [14]:
def processSnapshot(fname):
    import xml.etree.ElementTree as ET
    import statistics as st
    import sqlite3
    import os
    import time
    from pathlib import Path

    def mstime():
        return round(time.time() * 1000)
    def sync():
        os.system('sync')
    def touch(path):
        with open(path, 'a'):
            os.utime(path, None)
        sync()
    
    log = []
    p = Path(fname)
    baseDir = str(p.parent)
    dbName = "%s/%s.%s.db" % (tmpFolder, str(p.name), mstime())
    log.append("process %s. base: %s, tmp database: %s" % (fname, baseDir, dbName))
    def processShop(tree, time):
        shopData = []
        ttidList = []
        for good in tree:
            goodData = {}
            goodData["ttid"] = int(good.findall("ttid")[0].text)
            ttidList.append(goodData["ttid"])
            goodData["time"] = time
            goodData["shop"] = tree.attrib["name"]
            goodData["shopid"] = tree.attrib["id"]
            goodData["shopmoney"] = tree.attrib["money"]
            sellElems = good.findall("sell")
            if sellElems:
                goodData["sell"] = float(sellElems[0].text)
            else:
                goodData["sell"] = 0.0

            buyElems = good.findall("buy")
            if buyElems:
                goodData["buy"] = float(buyElems[0].text)
            else:
                goodData["buy"] = 0.0
            goodData["count"] = int(good.findall("count")[0].text)
            goodData["thingname"] = good.findall("thingname")[0].text
            shopData.append(goodData)
        return (shopData, ttidList)
    

    if os.path.exists(dbName):
        os.remove(dbName)
    sync()
    touch(dbName)
    timetag = "lastupdatetime"
    shopTag = "shop"
    xmltree = ET.parse(fname)
    xmlroot = xmltree.getroot()
    timeelem = xmlroot.find(timetag)
    dataTime = int(timeelem.text)
    orders = []
    ttids = []
    con = sqlite3.connect(dbName)    
    cur = con.cursor()
    q = dbquery_cratetable.replace("\n", " ")
    cur.execute(q)
    for shop in xmlroot.findall(shopTag):
        rc = processShop(shop, dataTime)
        shopData = rc[0]
        ttidList = rc[1]
        orders.extend(shopData)
        ttids.extend(ttidList)
    ttid_summary = []
    for ttid in ttids:
        ttid_stat = {}
        ttid_stat["ttid"] = ttid
        ttid_stat["time"] = dataTime
        ttid_stat["buy"] = []
        ttid_stat["sell"] = []
        ttid_stat["capacity"] = 0
        for order in orders:
            if order["ttid"] == ttid:
                ttid_stat["capacity"] += order["count"]
                if order["buy"] > 0:
                    ttid_stat["buy"].append(order["buy"] )
                if order["sell"] > 0:
                    ttid_stat["sell"].append(order["sell"] )  
        
        ttid_stat["buystat"] = {}
        if ttid_stat["buy"]:
            ttid_stat["buystat"]["avg"] = st.mean(ttid_stat["buy"])
            ttid_stat["buystat"]["min"] = min(ttid_stat["buy"])
            ttid_stat["buystat"]["max"] = max(ttid_stat["buy"])
            ttid_stat["buystat"]["gap"] = ttid_stat["buystat"]["max"] - ttid_stat["buystat"]["min"]
            ttid_stat["buystat"]["median"] = st.median(ttid_stat["buy"])
            ttid_stat["buystat"]["stdev"] = 0
            if len(ttid_stat["buy"]) >= 2:
                ttid_stat["buystat"]["stdev"] = st.stdev(ttid_stat["buy"], ttid_stat["buystat"]["avg"])
        else:
            ttid_stat["buystat"]["avg"] = 0.0
            ttid_stat["buystat"]["min"] = 0.0
            ttid_stat["buystat"]["max"] = 0.0
            ttid_stat["buystat"]["gap"] = 0.0
            ttid_stat["buystat"]["median"] = 0.0
            ttid_stat["buystat"]["stdev"] = 0.0
            
        ttid_stat["sellstat"] = {}
        if len(ttid_stat["sell"]) >= 2:
            ttid_stat["sellstat"]["avg"] = st.mean(ttid_stat["sell"])
            ttid_stat["sellstat"]["min"] = min(ttid_stat["sell"])
            ttid_stat["sellstat"]["max"] = max(ttid_stat["sell"])
            ttid_stat["sellstat"]["gap"] = ttid_stat["sellstat"]["max"] - ttid_stat["sellstat"]["min"]
            ttid_stat["sellstat"]["median"] = st.median(ttid_stat["sell"])
            ttid_stat["sellstat"]["stdev"] = 0
            if len(ttid_stat["sell"]) >= 2:
                ttid_stat["sellstat"]["stdev"] = st.stdev(ttid_stat["sell"], ttid_stat["sellstat"]["avg"])
        else:
            ttid_stat["sellstat"]["avg"] = 0.0
            ttid_stat["sellstat"]["min"] = 0.0
            ttid_stat["sellstat"]["max"] = 0.0
            ttid_stat["sellstat"]["median"] = 0.0
            ttid_stat["sellstat"]["gap"] = 0.0
            ttid_stat["sellstat"]["stdev"] = 0.0
        
        ttid_summary.append(ttid_stat)
        q = dbquery_insert.replace("\n", " ")
        try:
            cur.execute(q, (ttid_stat["time"],
                                            ttid_stat["ttid"],
                                            str(ttid_stat["buy"]),
                                            str(ttid_stat["sell"]),
                                            ttid_stat["capacity"],
                                            ttid_stat["buystat"]["avg"],
                                            ttid_stat["buystat"]["min"],
                                            ttid_stat["buystat"]["max"],
                                            ttid_stat["buystat"]["median"],
                                            ttid_stat["buystat"]["gap"],
                                            ttid_stat["buystat"]["stdev"],
                                            ttid_stat["sellstat"]["avg"],
                                            ttid_stat["sellstat"]["min"],
                                            ttid_stat["sellstat"]["max"],
                                            ttid_stat["sellstat"]["median"],
                                            ttid_stat["sellstat"]["gap"],
                                            ttid_stat["sellstat"]["stdev"]))
        except Exception as e:
            log.append("db exception %s in query %s" % (q, str(e)))
            return (None, fname, log)
    
    con.commit()
    con.close()
    sync()
    return (dbName, fname, log)

In [15]:
%%time
cleanupTmp()
sync()
full_pathes = [("%s/%s" % (os.getcwd(), f)) for f in data_sources]

def syncExec(pathes):
    parts = []
    counter = 0
    for p in pathes:
        print("%s - try (%d/%d)" % (p, counter, len(full_pathes)))
        parts.append(processSnapshot(p))
        print("%s - done (%d/%d)" % (p, counter, len(full_pathes)))
        sys.stdout.flush()
        counter += 1
    return parts
parts = workers.map_async(processSnapshot, full_pathes).get()
#parts = syncExec(full_pathes)
print("data size %s" % size(asizeof.asizeof(parts)))


clean tmp in /tmp
data size 1M
CPU times: user 127 ms, sys: 48.5 ms, total: 176 ms
Wall time: 31min 14s


In [16]:
#%%time
import shutil
def mprint(msg: str):
    print(msg)
    sys.stdout.flush()
   

outCon = sqlite3.connect(midDbTmp)
outcur = outCon.cursor()

backupDb = "%s/midData/mid_BU%d.db" % (os.getcwd(), mstime())
if os.path.isfile(midDbTmp):
    shutil.copyfile(midDbTmp, backupDb)
    sync()
mprint("midDbExample %s " % cutPathes(midDbExample))
mprint("midDb %s " % cutPathes(midDbTmp))
shutil.copyfile(midDbExample, midDbTmp)  
sync()

totalMerged = 0
partsCount = len(parts)
partsIndex = 0
for part in parts:
    partstart = time.time()
    if not part[0]:
        print("cannot process %s" % part[1])
        for logline in part[2]:
            print(logline)
        break
    dbName = part[0]
    mprint("process %s. (part %d/%d)" % (cutPathes(dbName), partsIndex, partsCount))
    secondDbName = 'db' + str(partsCount) + str(mstime())
    outcur.execute("ATTACH DATABASE ? as ? ;", (dbName, secondDbName))
    column_names="shapshotid,ttid,buy_list,sell_list,capacity,buystat_avg,buystat_min,buystat_max,buystat_median,buystat_gap,buystat_stdev,sellstat_avg,sellstat_min,sellstat_max,sellstat_median,sellstat_gap,sellstat_stdev"
    q = "INSERT INTO ttids (%s) SELECT %s FROM %s.ttids;" % (column_names, column_names, secondDbName)
    outcur.execute(q)
    outCon.commit()
    outcur.execute("DETACH  DATABASE %s ;" % secondDbName)
    partdone = time.time()
    partelapsed = partdone - partstart
    mprint("done -- %d parts merged (of %d). total %d. time %s " % (partsIndex, partsCount, totalMerged, str(partelapsed)))
    partsIndex += 1
    
outCon.commit()
q = "SELECT COUNT(ttid) FROM ttids"
outcur.execute(q)
print("done. records in mid db: %d" % outcur.fetchall()[0][0])
outCon.close()
shutil.copyfile(midDbTmp, midDb)

midDbExample midData/mid.example.db 
midDb midtmp.1633961083966.db 
process /tmp/1628517800_shops.xml.1633961084495.db. (part 0/2751)
done -- 0 parts merged (of 2751). total 0. time 0.058942556381225586 
process /tmp/1628519402_shops.xml.1633961087870.db. (part 1/2751)
done -- 1 parts merged (of 2751). total 0. time 0.046021223068237305 
process /tmp/1628521201_shops.xml.1633961090941.db. (part 2/2751)
done -- 2 parts merged (of 2751). total 0. time 0.10969829559326172 
process /tmp/1628523001_shops.xml.1633961094210.db. (part 3/2751)
done -- 3 parts merged (of 2751). total 0. time 0.03769278526306152 
process /tmp/1628526601_shops.xml.1633961096996.db. (part 4/2751)
done -- 4 parts merged (of 2751). total 0. time 0.04863119125366211 
process /tmp/1628528401_shops.xml.1633961099728.db. (part 5/2751)
done -- 5 parts merged (of 2751). total 0. time 0.09598159790039062 
process /tmp/1628530201_shops.xml.1633961102470.db. (part 6/2751)
done -- 6 parts merged (of 2751). total 0. time 0.0807

done -- 59 parts merged (of 2751). total 0. time 0.02854633331298828 
process /tmp/1628629201_shops.xml.1633961229390.db. (part 60/2751)
done -- 60 parts merged (of 2751). total 0. time 0.03636431694030762 
process /tmp/1628631001_shops.xml.1633961231668.db. (part 61/2751)
done -- 61 parts merged (of 2751). total 0. time 0.02884960174560547 
process /tmp/1628632801_shops.xml.1633961233975.db. (part 62/2751)
done -- 62 parts merged (of 2751). total 0. time 0.023294687271118164 
process /tmp/1628634602_shops.xml.1633961235799.db. (part 63/2751)
done -- 63 parts merged (of 2751). total 0. time 0.03306007385253906 
process /tmp/1628636401_shops.xml.1633961237861.db. (part 64/2751)
done -- 64 parts merged (of 2751). total 0. time 0.02948594093322754 
process /tmp/1628638201_shops.xml.1633961239465.db. (part 65/2751)
done -- 65 parts merged (of 2751). total 0. time 0.025784969329833984 
process /tmp/1628640001_shops.xml.1633961240999.db. (part 66/2751)
done -- 66 parts merged (of 2751). tota

process /tmp/1628735401_shops.xml.1633961343088.db. (part 119/2751)
done -- 119 parts merged (of 2751). total 0. time 0.023964643478393555 
process /tmp/1628737201_shops.xml.1633961343725.db. (part 120/2751)
done -- 120 parts merged (of 2751). total 0. time 0.02261495590209961 
process /tmp/1628739001_shops.xml.1633961344445.db. (part 121/2751)
done -- 121 parts merged (of 2751). total 0. time 0.019243478775024414 
process /tmp/1628742601_shops.xml.1633961345244.db. (part 122/2751)
done -- 122 parts merged (of 2751). total 0. time 0.0321049690246582 
process /tmp/1628744401_shops.xml.1633961346509.db. (part 123/2751)
done -- 123 parts merged (of 2751). total 0. time 0.03220534324645996 
process /tmp/1628746201_shops.xml.1633961347633.db. (part 124/2751)
done -- 124 parts merged (of 2751). total 0. time 0.03747367858886719 
process /tmp/1628748001_shops.xml.1633961349629.db. (part 125/2751)
done -- 125 parts merged (of 2751). total 0. time 0.04178786277770996 
process /tmp/1628749802_sh

process /tmp/1628847001_shops.xml.1633961456448.db. (part 178/2751)
done -- 178 parts merged (of 2751). total 0. time 0.029244184494018555 
process /tmp/1628848801_shops.xml.1633961458703.db. (part 179/2751)
done -- 179 parts merged (of 2751). total 0. time 0.028645992279052734 
process /tmp/1628850601_shops.xml.1633961461137.db. (part 180/2751)
done -- 180 parts merged (of 2751). total 0. time 0.03881335258483887 
process /tmp/1628852401_shops.xml.1633961463309.db. (part 181/2751)
done -- 181 parts merged (of 2751). total 0. time 0.03490447998046875 
process /tmp/1628854201_shops.xml.1633961465403.db. (part 182/2751)
done -- 182 parts merged (of 2751). total 0. time 0.03816795349121094 
process /tmp/1628856001_shops.xml.1633961467608.db. (part 183/2751)
done -- 183 parts merged (of 2751). total 0. time 0.041140079498291016 
process /tmp/1628857801_shops.xml.1633961470031.db. (part 184/2751)
done -- 184 parts merged (of 2751). total 0. time 0.04091620445251465 
process /tmp/1628859601_

process /tmp/1628955001_shops.xml.1633961600872.db. (part 237/2751)
done -- 237 parts merged (of 2751). total 0. time 0.05359697341918945 
process /tmp/1628958601_shops.xml.1633961604501.db. (part 238/2751)
done -- 238 parts merged (of 2751). total 0. time 0.05063891410827637 
process /tmp/1628960401_shops.xml.1633961607169.db. (part 239/2751)
done -- 239 parts merged (of 2751). total 0. time 0.04020380973815918 
process /tmp/1628962201_shops.xml.1633961609993.db. (part 240/2751)
done -- 240 parts merged (of 2751). total 0. time 0.040724754333496094 
process /tmp/1628964001_shops.xml.1633961612720.db. (part 241/2751)
done -- 241 parts merged (of 2751). total 0. time 0.029774904251098633 
process /tmp/1628965801_shops.xml.1633961615358.db. (part 242/2751)
done -- 242 parts merged (of 2751). total 0. time 0.04244542121887207 
process /tmp/1628967601_shops.xml.1633961618084.db. (part 243/2751)
done -- 243 parts merged (of 2751). total 0. time 0.039000511169433594 
process /tmp/1628969401_

...

process /tmp/1633458601_shops.xml.1633962574360.db. (part 2576/2751)
done -- 2576 parts merged (of 2751). total 0. time 0.03108382225036621 
process /tmp/1633460401_shops.xml.1633962578052.db. (part 2577/2751)
done -- 2577 parts merged (of 2751). total 0. time 0.07639455795288086 
process /tmp/1633462201_shops.xml.1633962581841.db. (part 2578/2751)
done -- 2578 parts merged (of 2751). total 0. time 0.048226356506347656 
process /tmp/1633464001_shops.xml.1633962585106.db. (part 2579/2751)
done -- 2579 parts merged (of 2751). total 0. time 0.044634342193603516 
process /tmp/1633465801_shops.xml.1633962588483.db. (part 2580/2751)
done -- 2580 parts merged (of 2751). total 0. time 0.04197239875793457 
process /tmp/1633467601_shops.xml.1633962591011.db. (part 2581/2751)
done -- 2581 parts merged (of 2751). total 0. time 0.047704458236694336 
process /tmp/1633469401_shops.xml.1633962593452.db. (part 2582/2751)
done -- 2582 parts merged (of 2751). total 0. time 0.04024982452392578 
process /t

done -- 2633 parts merged (of 2751). total 0. time 0.03973579406738281 
process /tmp/1633563001_shops.xml.1633962719748.db. (part 2634/2751)
done -- 2634 parts merged (of 2751). total 0. time 0.042768239974975586 
process /tmp/1633564802_shops.xml.1633962721274.db. (part 2635/2751)
done -- 2635 parts merged (of 2751). total 0. time 0.0350186824798584 
process /tmp/1633566601_shops.xml.1633962722867.db. (part 2636/2751)
done -- 2636 parts merged (of 2751). total 0. time 0.02396392822265625 
process /tmp/1633568401_shops.xml.1633962724174.db. (part 2637/2751)
done -- 2637 parts merged (of 2751). total 0. time 0.029010295867919922 
process /tmp/1633570201_shops.xml.1633962725470.db. (part 2638/2751)
done -- 2638 parts merged (of 2751). total 0. time 0.021628141403198242 
process /tmp/1633572001_shops.xml.1633962726657.db. (part 2639/2751)
done -- 2639 parts merged (of 2751). total 0. time 0.03746843338012695 
process /tmp/1633573801_shops.xml.1633962727861.db. (part 2640/2751)
done -- 264

done -- 2691 parts merged (of 2751). total 0. time 0.052102088928222656 
process /tmp/1633685402_shops.xml.1633962824384.db. (part 2692/2751)
done -- 2692 parts merged (of 2751). total 0. time 0.04298233985900879 
process /tmp/1633687201_shops.xml.1633962826985.db. (part 2693/2751)
done -- 2693 parts merged (of 2751). total 0. time 0.04186749458312988 
process /tmp/1633689001_shops.xml.1633962829444.db. (part 2694/2751)
done -- 2694 parts merged (of 2751). total 0. time 0.05073356628417969 
process /tmp/1633690801_shops.xml.1633962831801.db. (part 2695/2751)
done -- 2695 parts merged (of 2751). total 0. time 0.05297517776489258 
process /tmp/1633692601_shops.xml.1633962834372.db. (part 2696/2751)
done -- 2696 parts merged (of 2751). total 0. time 0.0497736930847168 
process /tmp/1633694401_shops.xml.1633962837033.db. (part 2697/2751)
done -- 2697 parts merged (of 2751). total 0. time 0.05782508850097656 
process /tmp/1633696201_shops.xml.1633962839660.db. (part 2698/2751)
done -- 2698 

done -- 2749 parts merged (of 2751). total 0. time 0.0313115119934082 
process /tmp/1633793401_shops.xml.1633962956685.db. (part 2750/2751)
done -- 2750 parts merged (of 2751). total 0. time 0.05290961265563965 
done. records in mid db: 11729563


'/home/ed/repo/github/pyTest/dataSet/haddan/midData/mid.db'